In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
%cd data

c:\Users\User\document\final_project\data


In [3]:
# 처방전 관련 데이터(2021년 3월 1000만개정도 처방받은 데이터)
df = pd.read_csv('data1.csv',encoding='cp949')
df = df[['가입자 일련번호', '처방내역일련번호', '일련번호', '성별코드', '연령대코드(5세단위)', '시도코드','약품일반성분명코드', '총투여일수', '단가', '금액']]

province_codes = {11: '서울특별시',26: '부산광역시',27: '대구광역시',28: '인천광역시',29: '광주광역시',30: '대전광역시',31: '울산광역시',36: '세종특별자치시',
    41: '경기도',42: '강원도',43: '충청북도',44: '충청남도',45: '전라북도',46: '전라남도',47: '경상북도',48: '경상남도',49: '제주특별자치도'}

df['시도명'] = df['시도코드'].map(province_codes)
df['시도명']

KeyboardInterrupt: 

In [4]:
df.head()

,가입자 일련번호,처방내역일련번호,일련번호,성별코드,연령대코드(5세단위),시도코드,약품일반성분명코드,총투여일수,단가,금액,시도명
0,666668,5260626,2,1,11,43,134103ATB,4,476.0,7616,충청북도
1,666668,6859861,1,1,11,43,146432ASS,3,14.0,1680,충청북도
2,666668,769595,1,1,11,43,451302ATB,7,53.0,186,충청북도
3,666668,6859861,4,1,11,43,185102ACH,3,30.0,90,충청북도
4,666668,9307313,2,1,11,43,260100ACR,3,302.0,906,충청북도


In [5]:
# 약품별 분류번호등 정보 
df2 = pd.read_csv('data2.csv',encoding='cp949')

In [6]:
df2.head()

,일반명코드,제형구분코드,제형,일반명,분류번호,투여,함량,단위
0,100101AGN,GN,"과립제,세립",상황균사체엑스,421,내복,1.1,g
1,100101ALQ,LQ,액제,상황균사체엑스,421,내복,1.1,g
2,100101APD,PD,산제,phellinus linteus mycelliun ext.,421,내복,1.1,g
3,100102ACH,CH,경질캡슐제,phellinus linteus mycelliun ext.,421,내복,0.55,g
4,100102ALQ,LQ,액제,phellinus linteus mycelliun ext.,421,내복,1.1,g(55mg/mL)


In [7]:
# 진료 내역정보 (환자가 어떤 증상으로 약을 처방 받았는지)
df3 = pd.read_csv('data3.csv',encoding='cp949')

In [8]:
# 기호별 병 정보
df4 = pd.read_csv('data4.csv',encoding='cp949')
df4 = df4[['상병기호','한글명','영문영']]
df4.drop_duplicates(subset='상병기호', inplace=True) # 상병 중복제거
df4['상병기호']

0         A00
1        A000
3        A001
5        A009
6         A01
         ... 
47791    Z992
47794    Z993
47795    Z994
47796    Z998
47797    Z999
Name: 상병기호, Length: 21299, dtype: object

In [9]:
df_new = pd.merge(df, df2, left_on = '약품일반성분명코드', right_on ='일반명코드', how = 'left')  
df_new2 = pd.merge(df3, df4, left_on = '주상병코드', right_on ='상병기호', how = 'left') # (진료)기호별로 병이름을 알기위해 합침

In [10]:
df_new.isnull().sum()
df_new.dropna(inplace=True)

def age_level(age):
    if age == 1 or age == 2:
        result = '9세미만'
    elif age == 3 or age == 4:
        result = '10대'
    elif age == 5 or age == 6:
        result = '20대'
    elif age == 7 or age == 8:
        result = '30대'
    elif age == 9 or age == 10:
        result = '40대'
    elif age == 11 or age == 12:
        result = '50대'
    elif age == 13 or age == 14:
        result = '60대'
    elif age == 15 or age == 16:
        result = '70대'
    else:
        result = '80대'
    return result
        
df_new['연령대'] = df_new['연령대코드(5세단위)'].map(lambda x: age_level(x))
# df_new['연령대'].value_counts()

df_new = df_new[df_new['투여']=='내복 ']
df_new['분류번호'] = df_new['분류번호'].astype(int)
df_new.drop(['처방내역일련번호','일련번호','시도코드','약품일반성분명코드','제형구분코드','투여'],axis=1,inplace=True)
# df_new.groupby('일반명')['연령대'].value_counts() # 약품 종류가 6915개 / 연령대별로 어떻게 많이 먹는지 확인
df_new.head(3)

,가입자 일련번호,성별코드,연령대코드(5세단위),총투여일수,단가,금액,시도명,일반명코드,제형,일반명,분류번호,함량,단위,연령대
0,666668,1,11,4,476.0,7616,충청북도,134103ATB,"정제,저작정",ciprofloxacin hydrochloride (as ciprofloxacin),629,0.25,g,50대
1,666668,1,11,3,14.0,1680,충청북도,146432ASS,"현탁액제,현탁액용정제,현탁액용산제",dioctahedral smectite,237,75,g(0.15g/mL),50대
2,666668,1,11,7,53.0,186,충청북도,451302ATB,"정제,저작정",chlorthalidone,213,25,mg,50대


In [ ]:
extracted_data= agelevel_name.apply(lambda x: x.head(3)) # 연령대별로 약별로 가장 많이 처방많이 받은약 3개씩 추출
extracted_data

In [11]:
df_new2 = df_new2[['가입자 일련번호','진료내역일련번호','성별코드','연령대코드','시도코드','진료과목코드','상병기호','한글명','영문영']]
df_new2.head(3)

,가입자 일련번호,진료내역일련번호,성별코드,연령대코드,시도코드,진료과목코드,상병기호,한글명,영문영
0,666847,3243108,1,14,41,1,I109,기타 및 상세불명의 원발성 고혈압,Other and unspecified primary hypertension
1,666847,5512427,1,14,41,1,I109,기타 및 상세불명의 원발성 고혈압,Other and unspecified primary hypertension
2,666847,4023485,1,14,41,1,I109,기타 및 상세불명의 원발성 고혈압,Other and unspecified primary hypertension


In [4]:
data = pd.read_csv('df_merged.csv',encoding='utf-8')
data.head(3)

,idx,file_name,width,height,imgfile,drug_N,drug_S,back_color,drug_dir,light_color,camera_la,camera_lo,size,dl_idx,dl_mapping_code,dl_name,dl_name_en,img_key,dl_material,dl_material_en,dl_custom_shape,item_seq,di_item_permit_date,di_class_no,di_etc_otc_code,di_edi_code,drug_shape,thick,leng_long,leng_short,print_front,print_back,color_class1,img_regist_ts,form_code_name,change_date,area,iscrowd,bbox
0,0,K-019461_0_0_0_0_60_000_200.png,976,1280,K-019461_0_0_0_0_60_000_200.png,K-019461,정상알약,검은색 배경,앞면,전구색,60,0,200,19460,K-019461,미래트리메부틴정 100mg/병,Trimebutine Tab. Mirae,http://connectdi.com/design/img/drug/147427567...,트리메부틴말레산염,Trimebutine,"정제, 저작정",200607006,20060724,[02390]기타의 소화기관용약,일반의약품,657500470,원형,4.25,9.05,9.05,NaN,마크TM,연두,20110516.0,나정,20200930.0,50838,0,"[408, 519, 229, 222]"
1,1,K-019469_0_0_0_0_60_000_200.png,976,1280,K-019469_0_0_0_0_60_000_200.png,K-019469,정상알약,검은색 배경,앞면,전구색,60,0,200,19468,K-019469,큐레틴정(빌베리건조엑스),Qretin Tab.,http://connectdi.com/design/img/drug/147428234...,빌베리건조엑스,Bilberry Dried Ext.,"정제, 저작정",200607018,20061009,[01310]안과용제,일반의약품,"650101140,A28302561",타원형,5.65,12.84,7.60,QRT,분할선,갈색,20070330.0,필름코팅정,20130129.0,72657,0,"[350, 490, 299, 243]"
2,2,K-019553_0_0_0_0_60_000_200.png,976,1280,K-019553_0_0_0_0_60_000_200.png,K-019553,정상알약,검은색 배경,앞면,전구색,60,0,200,19552,K-019553,콘택골드캡슐 10mg/PTP,Contac Gold Cap.,http://connectdi.com/design/img/drug/147427262...,클로르페니라민말레산염|벨라돈나총알칼로이드|페닐레프린염산염,Phenylephrine Hydrochloride| Belladonna Total ...,경질캡슐제,200607735,20060728,[01410]항히스타민제,일반의약품,642102330,장방형,6.77,19.08,6.42,YH CNT,NaN,빨강,20071113.0,"서방성캡슐제, 펠렛",20201116.0,126169,0,"[249, 406, 449, 281]"


In [7]:
data.columns

Index(['idx', 'file_name', 'width', 'height', 'imgfile', 'drug_N', 'drug_S',
       'back_color', 'drug_dir', 'light_color', 'camera_la', 'camera_lo',
       'size', 'dl_idx', 'dl_mapping_code', 'dl_name', 'dl_name_en', 'img_key',
       'dl_material', 'dl_material_en', 'dl_custom_shape', 'item_seq',
       'di_item_permit_date', 'di_class_no', 'di_etc_otc_code', 'di_edi_code',
       'drug_shape', 'thick', 'leng_long', 'leng_short', 'print_front',
       'print_back', 'color_class1', 'img_regist_ts', 'form_code_name',
       'change_date', 'area', 'iscrowd', 'bbox'],
      dtype='object')

In [8]:
data.isnull().sum()

idx                       0
file_name                 0
width                     0
height                    0
imgfile                   0
drug_N                    0
drug_S                    0
back_color                0
drug_dir                  0
light_color               0
camera_la                 0
camera_lo                 0
size                      0
dl_idx                    0
dl_mapping_code           0
dl_name                   0
dl_name_en               87
img_key                  49
dl_material              27
dl_material_en          176
dl_custom_shape         136
item_seq                  0
di_item_permit_date       0
di_class_no               2
di_etc_otc_code           0
di_edi_code              99
drug_shape               49
thick                    22
leng_long                21
leng_short               23
print_front              90
print_back             2099
color_class1             49
img_regist_ts            49
form_code_name           49
change_date         

In [28]:
data['image_id'].value_counts()

image_id
1    4024
Name: count, dtype: int64

In [6]:
data = data[['file_name','drug_N','back_color','dl_idx','dl_name','dl_name_en','dl_material','dl_material_en',
              'dl_custom_shape','dl_company','dl_company_en','di_class_no','item_seq','di_etc_otc_code','di_edi_code','chart',
              'drug_shape','leng_long','leng_short','print_front','print_back','line_front','line_back','color_class1','form_code_name','area','bbox']]

KeyError: "['dl_company', 'dl_company_en', 'chart', 'line_front', 'line_back'] not in index"

In [34]:
data.to_csv('merge_data3.csv')

In [47]:
keys = ['file_name','drug_N','dl_idx','dl_name','dl_name_en','dl_material','dl_material_en','dl_company','dl_company_en','di_class_no','item_seq','di_etc_otc_code','di_edi_code','chart',
'drug_shape','print_front','print_back','color_class1','form_code_name']
values = ['파일이름','알약코드','알약 식별자','제품명','제품명(영어)','성분명','성분명(영어)','제조사명','제조사명(영어)','약품 분류','품목기준 코드','전문의약품 여부','EDI코드','제형',
'모양','식별문자(앞)','식별문자(뒤)','색상','정제 분류명','영역']
my_dict = dict(zip(keys, values))
data = data.rename(columns=my_dict)
data['분류코드'] = data['약품 분류'].str[2:5]

In [74]:
data.head(2)

,파일이름,알약코드,알약 식별자,제품명,제품명(영어),성분명,성분명(영어),제조사명,제조사명(영어),약품 분류,품목기준 코드,전문의약품 여부,EDI코드,제형,모양,식별문자(앞),식별문자(뒤),색상,정제 분류명,area,분류코드
0,K-019461_0_0_0_0_60_000_200.png,K-019461,19460,미래트리메부틴정 100mg/병,Trimebutine Tab. Mirae,트리메부틴말레산염,Trimebutine,미래제약(주),Mirae Pharm Korea,[02390]기타의 소화기관용약,200607006,일반의약품,657500470,연두색의 원형정제,원형,NaN,마크TM,연두,나정,50838,239
1,K-019469_0_0_0_0_60_000_200.png,K-019469,19468,큐레틴정(빌베리건조엑스),Qretin Tab.,빌베리건조엑스,Bilberry Dried Ext.,(주)태준제약,Taejoon Pharm,[01310]안과용제,200607018,일반의약품,"650101140,A28302561",적갈색의 타원형 필름코팅정,타원형,QRT,분할선,갈색,필름코팅정,72657,131


In [72]:
data[data['분류코드']=='114']['성분명']

6                                아세트아미노펜|이소프로필안티피린|카페인무수물
16                                         클로르족사존|아세트아미노펜
23                                              이부프로펜아르기닌
38                                                 덱시부프로펜
39                      금은화|연교|두시|우방자|박하|길경|감초|담죽엽|형개|영양각
                              ...                        
3979    아세트아미노펜|구아이페네신|슈도에페드린염산염|클로페라스틴염산염|dl-메틸에페드린염산...
3980                                       아세트아미노펜|카페인수화물
3985                                              아세트아미노펜
3999                                       클로르족사존|아세트아미노펜
4022    아세트아미노펜|에텐자미드|티페피딘히벤즈산염|dl-메틸에페드린염산염|클로르페니라민말레...
Name: 성분명, Length: 488, dtype: object

In [66]:
data[data['성분명(영어)'].str.contains('Triazolam')==True]['성분명(영어)']

Series([], Name: 성분명(영어), dtype: object)

In [28]:
multi_no = pd.read_excel('multino.xlsx')
multi_no = multi_no[['Unnamed: 1','Unnamed: 2','Unnamed: 3']]
multi_no = multi_no.rename(columns={'Unnamed: 1':'유효성분1','Unnamed: 2':'유효성분2', 'Unnamed: 3':'위험증상'})

In [78]:
multi_no.isnull().sum()
multi_no.dropna(subset='위험증상',inplace=True)
multi_no = multi_no[1:]
multi_no

,유효성분1,유효성분2,위험증상
1,acemetacin,diflunisal,중증의 위장관계 이상반응
2,acemetacin,methotrexate(분류번호 : 421),신세뇨관에서 Methotrexate 배설이 지연되어 Methotrexate의 혈액학...
3,acetazolamide,methazolamide,신장결석 형성 위험 증가
4,acetazolamide,topiramate,신결석
5,acitretin,doxycycline,두개내압 증가
...,...,...,...
1152,Doravirine + Lamivudine + Tenofovir disoproxil,Adefovir dipivoxil,뉴클레오시드 유사체 단독투여 또는 다른 항레트로바이러스제를 병용투여했을 때 치명적인...
1153,Triazolam,Ritonavir,"진정작용의 연장 또는 증가, 호흡 저하"
1154,Triazolam,Ritonavir + Lopinavir,"진정작용의 연장 또는 증가, 호흡 저하"
1155,Irinotecan,St. john's wort,이리노테칸 활성 대사물(SN-38)의 혈중 농도가 저하되어 작용 감소


In [285]:
# 문제될수있는 성분을 포함하는 이미지를 추출해줌 
index_1 = data[data['성분명(영어)'].isin(multi_no['유효성분1'].values)]['성분명(영어)'].index # 문제될수 있는 성분1이 있는 index
index_2 = data[data['성분명(영어)'].isin(multi_no['유효성분2'].values)]['성분명(영어)'].index # 문제될수 있는 성분2이 있는 index 
# data['New Variable'] = data.index.isin(index_values).map(lambda x: True if x else False)

data['문제가능'] = None
data.loc[index_1,'문제가능'] = 'problem'
data.loc[index_2,'문제가능'] = 'problem'

In [289]:
data[data['문제가능']=='problem']['문제가능'] # 유효성분을 하나라도 포함하는 index만 추출 

31      problem
151     problem
177     problem
545     problem
615     problem
676     problem
688     problem
708     problem
710     problem
711     problem
712     problem
714     problem
716     problem
718     problem
776     problem
813     problem
827     problem
974     problem
1008    problem
1017    problem
1018    problem
1332    problem
1426    problem
1443    problem
1462    problem
1565    problem
1654    problem
1669    problem
1678    problem
1896    problem
2175    problem
2482    problem
2734    problem
2740    problem
2777    problem
3692    problem
3930    problem
3954    problem
Name: 문제가능, dtype: object

In [309]:
data['성분명(영어)'].str.contains('Haloperidol').sum()

5

In [164]:
multi_no[multi_no['유효성분1'].isin(data['성분명(영어)'])] # 문제되는 조합 
    

,유효성분1,유효성분2,위험증상
1082,Simvastatin,Cyclosporine,혈중농도 상승 및 근육병증/횡문근융해증 위험성 증가
1094,Haloperidol,Gemifloxacin,QT간격 연장 위험 증가


In [165]:
multi_no[multi_no['유효성분2'].isin(data['성분명(영어)'])] # 문제되는 조합
    

,유효성분1,유효성분2,위험증상
1081,Atorvastatin,Cyclosporine,혈중농도 상승 및 근육병증/횡문근융해증 위험성 증가
1082,Simvastatin,Cyclosporine,혈중농도 상승 및 근육병증/횡문근융해증 위험성 증가
1083,Rosuvastatin,Cyclosporine,혈중농도 상승 및 근육병증/횡문근융해증 위험성 증가
1084,Lovastatin,Cyclosporine,혈중농도 상승 및 근육병증/횡문근융해증 위험성 증가
1085,Pitavastatin,Cyclosporine,혈중농도 상승 및 근육병증/횡문근융해증 위험성 증가
1098,Gimeracil + Oteracil + Tegafur,Capecitabine,해당 약물 병용 시 gimeracil에 의해 fluoropyrimidine 계열 약...
1100,Sulfamethoxazole + Trimethoprim,Methotrexate,신세뇨관에서 methotrexate 배설이 지연되어 methotrexate의 혈액학...
1103,Glecaprevir + Pibrentasvir,Simvastatin,횡문근융해증을 비롯한 근육질환 등 중증 이상반응
1104,Glecaprevir + Pibrentasvir,Carbamazepine,Glecaprevir와 Pibrentasvir의 혈장 농도가 유의하게 감소하여 이 ...
1105,Glecaprevir + Pibrentasvir,Phenytoin,Glecaprevir와 Pibrentasvir의 혈장 농도가 유의하게 감소하여 이 ...


In [317]:
data[data['성분명(영어)'] 'Haloperidol']

,파일이름,알약코드,알약 식별자,제품명,제품명(영어),성분명,성분명(영어),제조사명,제조사명(영어),약품 분류,품목기준 코드,전문의약품 여부,EDI코드,제형,모양,식별문자(앞),식별문자(뒤),색상,정제 분류명,area,분류코드,유효1,유효2,문제가능


In [292]:
no_list2 = multi_no[(data['성분명(영어)'].isin(multi_no['유효성분1'])) & (data['성분명(영어)'].isin(multi_no['유효성분2']))] 
no_list2 # 사진 성분이 유효성분을 포함하고 있을때 

C:\Users\User\AppData\Local\Temp\ipykernel_12872\2948727367.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  no_list2 = multi_no[(data['성분명(영어)'].isin(multi_no['유효성분1'])) & (data['성분명(영어)'].isin(multi_no['유효성분2']))]


,유효성분1,유효성분2,위험증상
31,amiloride,spironolactone,고칼륨혈증 위험 증가
